<a href="https://colab.research.google.com/github/Vsevolod245/ML-Tasks/blob/master/ML_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Лабораторная работа №3
##Выполнить реализацию GD в Numba и сравнить с реализацией в ЛР №1

In [ ]:
import numpy as np
import numba as nb
from numba import jit
from datetime import datetime
from typing import Tuple, Mapping
import math

Функция Шаффера №2

In [ ]:
# Функция Шаффера №2
def Sh2_E(x,y):
  return 0.5+((math.sin(x**2-y**2))**2-0.5)/(1+0.001*(x**2+y**2))**2
# Производная по x
def Sh2_dEdx(x,y):
  return (x*(-0.004*(math.sin(x**2-y**2))**2+(0.004*x**2+0.004*y**2+4)*math.sin(x**2-y**2)*math.cos(x**2-y**2)+0.002))/(0.001*(x**2+y**2)+1)**3
# Производная по y
def Sh2_dEdy(x,y):
  return (y*(-0.004*(math.sin(x**2-y**2))**2+(-0.004*x**2-0.004*y**2-4)*math.sin(x**2-y**2)*math.cos(x**2-y**2)+0.002))/(0.001*(x**2+y**2)+1)**3

# NUMBA

# Функция Шаффера №2 
@nb.njit(fastmath=True)
def NB_Sh2_E(x: np.ndarray):
  return [0.5+((math.sin(x[0]**2-x[1]**2))**2-0.5)/(1+0.001*(x[0]**2+x[1]**2))**2]
# Производная по x
@nb.njit(fastmath=True)
def NB_Sh2_dEdx(x: np.ndarray):
  return (x[0]*(-0.004*(math.sin(x[0]**2-x[1]**2))**2+(0.004*x[0]**2+0.004*x[1]**2+4)*math.sin(x[0]**2-x[1]**2)*math.cos(x[0]**2-x[1]**2)+0.002))/(0.001*(x[0]**2+x[1]**2)+1)**3
# Производная по y
@nb.njit(fastmath=True)
def NB_Sh2_dEdy(x: np.ndarray):
  return (x[1]*(-0.004*(math.sin(x[0]**2-x[1]**2))**2+(-0.004*x[0]**2-0.004*x[1]**2-4)*math.sin(x[0]**2-x[1]**2)*math.cos(x[0]**2-x[1]**2)+0.002))/(0.001*(x[0]**2+x[1]**2)+1)**3

Функция Шаффера №4

In [ ]:
# Функция Шаффера №4
def Sh4_E(x,y):
  return 0.5+((math.cos(math.sin(abs(x**2-y**2))))**2-0.5)/(1+0.001*(x**2+y**2))**2
# Производная по x
def Sh4_dEdx(x,y):
  return (x*(-0.004*((math.cos(math.sin(abs(x**2-y**2))))**2-0.5)-1/abs(x**2-y**2)*4*(x**2-y**2)*(0.001*(x**2+y**2)+1)*math.sin(math.sin(abs(x**2-y**2)))*math.cos(abs(x**2-y**2))*math.cos(math.sin(abs(x**2-y**2)))))/(0.001*(x**2+y**2)+1)**3
# Производная по y
def Sh4_dEdy(x,y):
  return (y*(1/abs(x**2-y**2)*4*(x**2-y**2)*(0.001*(x**2+y**2)+1)*math.sin(math.sin(abs(x**2-y**2)))*math.cos(abs(x**2-y**2))*math.cos(math.sin(abs(x**2-y**2)))-0.004*((math.cos(math.sin(abs(x**2-y**2))))**2-0.5)))/(0.001*(x**2+y**2)+1)**3

# NUMBA

#Функция Шаффера №4
@nb.njit(fastmath=True)
def NB_Sh4_E(x: np.ndarray):
  return 0.5+((math.cos(math.sin(abs(x[0]**2-x[1]**2))))**2-0.5)/(1+0.001*(x[0]**2+x[1]**2))**2
#Производная по x
@nb.njit(fastmath=True)
def NB_Sh4_dEdx(x: np.ndarray):
  return (x[0]*(-0.004*((math.cos(math.sin(abs(x[0]**2-x[1]**2))))**2-0.5)-1/abs(x[0]**2-x[1]**2)*4*(x[0]**2-x[1]**2)*(0.001*(x[0]**2+x[1]**2)+1)*math.sin(math.sin(abs(x[0]**2-x[1]**2)))*math.cos(abs(x[0]**2-x[1]**2))*math.cos(math.sin(abs(x[0]**2-x[1]**2)))))/(0.001*(x[0]**2+x[1]**2)+1)**3
#Производная по y
@nb.njit(fastmath=True)
def NB_Sh4_dEdy(x: np.ndarray):
  return (x[1]*(1/abs(x[0]**2-x[1]**2)*4*(x[0]**2-x[1]**2)*(0.001*(x[0]**2+x[1]**2)+1)*math.sin(math.sin(abs(x[0]**2-x[1]**2)))*math.cos(abs(x[0]**2-x[1]**2))*math.cos(math.sin(abs(x[0]**2-x[1]**2)))-0.004*((math.cos(math.sin(abs(x[0]**2-x[1]**2))))**2-0.5)))/(0.001*(x[0]**2+x[1]**2)+1)**3

Градиентный спуск

In [ ]:
def GD(E, dEdx, dEdy, x0, y0, LearningRate, Iterations):
  '''
  E - функция
  dEdx, dEdy - частные производные функции
  x0, y0 - начальные приближения точки
  LearningRate - скорость обучения
  Iterations - количество итераций
  '''
  x1=x0-LearningRate * dEdx(x0,y0)
  y1=y0-LearningRate * dEdy(x0,y0)
  for _ in range(Iterations):
    x0=x1
    y0=y1
    x1=x0-LearningRate * dEdx(x0,y0)
    y1=y0-LearningRate * dEdy(x0,y0)
  return x1,y1, E(x1,y1)

# NUMBA

@nb.njit(fastmath=True)
def NB_GD(E: Mapping, dEdx: Mapping, dEdy: Mapping, x:np.ndarray,LearningRate, Iterations):
  '''
  E - функция
  dEdx, dEdy - частные производные функции
  x - начальные приближения точки
  LearningRate - скорость обучения
  Iterations - количество итераций
  '''
  #Алгоритм градиентного спуска
  x0 = x[0];
  y0 = x[1];
  x1=x0-LearningRate * dEdx(np.array([x0, y0]))
  y1=y0-LearningRate * dEdy(np.array([x0, y0]))
  for _ in range(Iterations):
    x0=x1
    y0=y1
    x1=x0-LearningRate * dEdx(np.array([x0, y0]))
    y1=y0-LearningRate * dEdy(np.array([x0, y0]))
  return x1,y1, E(np.array([x1,y1]))

Тест

In [ ]:
print("###Градиентный спуск без NUMBA###")
print("___Функция Шаффера №2___")
start_time_without_NUMBA_Sh2 = datetime.now()
x, y, TestSh2 = GD(Sh2_E, Sh2_dEdx, Sh2_dEdy, -1, 1, 0.0001, 500000)
time_without_NUMBA_Sh2 = datetime.now() - start_time_without_NUMBA_Sh2
print("Время ", time_without_NUMBA_Sh2)
print('x={0}, y={1}, E={2}'.format(x,y,TestSh2))
print("================================================")
print("___Функция Шаффера №4___")
start_time_without_NUMBA_Sh4 = datetime.now()
x,y,TestSh4 = GD(Sh4_E, Sh4_dEdx, Sh4_dEdy, 0, 1, 0.001, 500000)
time_without_NUMBA_Sh4 = datetime.now() - start_time_without_NUMBA_Sh4
print("Время ", time_without_NUMBA_Sh4)
print('x={0}, y={1}, E={2}'.format(x,y,TestSh4))

print("\n###Градиентный спуск c NUMBA###")
print("___Функция Шаффера №2___")
start_time_with_NUMBA_Sh2 = datetime.now()
x,y,TestNB_Sh2 = NB_GD(NB_Sh2_E, NB_Sh2_dEdx, NB_Sh2_dEdy, np.array([-1, 1]), 0.0001, 500000)
time_with_NUMBA_Sh2 = datetime.now() - start_time_with_NUMBA_Sh2
print("Время:", time_with_NUMBA_Sh2)
print('x={0}, y={1}, E={2}'.format(x,y,TestNB_Sh2))
print("================================================")
print("___Функция Шаффера №4___")
start_time_with_NUMBA_Sh4 = datetime.now()
x,y,TestNB_Sh4 = NB_GD(NB_Sh4_E, NB_Sh4_dEdx, NB_Sh4_dEdy, np.array([0,1]), 0.001, 500000)
time_with_NUMBA_Sh4 = datetime.now() - start_time_with_NUMBA_Sh4
print("Время:", time_with_NUMBA_Sh4)
print('x={0}, y={1}, E={2}'.format(x,y,TestNB_Sh4))

###Градиентный спуск без NUMBA###
___Функция Шаффера №2___
Время  0:00:01.881756
x=-0.9053279000724023, y=0.9053279000724023, E=0.0016352153569050842
___Функция Шаффера №4___
Время  0:00:02.920717
x=0.0, y=1.2531318314637139, E=0.29257863203598056

###Градиентный спуск c NUMBA###
___Функция Шаффера №2___
Время: 0:00:00.117032
x=-0.9053279000724023, y=0.9053279000724023, E=[0.0016352153569050842]
___Функция Шаффера №4___
Время: 0:00:00.176648
x=0.0, y=1.2531318314637139, E=0.29257863203598056


Сравнение времени работы программы без NUMBA и с NUMBA

In [ ]:
print("Время работы программы для функции Шаффера №2 без NUMBA больше в {0} раз, по сравнению со временем работы с NUMBA.".format(round(time_without_NUMBA_Sh2/time_with_NUMBA_Sh2)))
print("Время работы программы для функции Шаффера №4 без NUMBA больше в {0} раз, по сравнению со временем работы с NUMBA.".format(round(time_without_NUMBA_Sh4/time_with_NUMBA_Sh4)))

Время работы программы для функции Шаффера №2 без NUMBA больше в 16 раз, по сравнению со временем работы с NUMBA.
Время работы программы для функции Шаффера №4 без NUMBA больше в 17 раз, по сравнению со временем работы с NUMBA.
